In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, data, labels=None, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx] if self.labels is not None else -1
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load ResNet-152 with weights, suppress warnings
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature_extractor = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])  # Remove final layer
feature_extractor.eval().to(device)  # Move model to device

Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:01<00:00, 170MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [3]:
def extract_features(model, dataloader, device='cpu'):
    """Extracts features for images using the feature extractor."""
    features, labels = [], []
    model.to(device)
    with torch.no_grad():
        for images, lbls in dataloader:
            images = images.to(device)
            feats = model(images).squeeze()  # Extract features
            features.append(feats.cpu())  # Store features on CPU to save GPU memory
            labels.append(lbls)
    return torch.cat(features, dim=0), torch.cat(labels, dim=0)

In [4]:
def compute_prototypes(features, labels, num_classes=10):
    """Computes class prototypes as the mean of feature vectors for each class."""
    prototypes = []
    for cls in range(num_classes):
        class_features = features[labels == cls]
        prototype = class_features.mean(dim=0)
        prototypes.append(prototype)
    return torch.stack(prototypes)

In [5]:
def classify_with_prototypes(prototypes, query_features):
    """Classifies query features by finding the nearest prototype."""
    distances = torch.cdist(query_features, prototypes)  # Pairwise distances
    return distances.argmin(dim=1)  # Nearest prototype index

In [7]:
import zipfile

# Path to the zip file
zip_file_path = '/content/eval_data.zip'

# Destination folder to extract the files
extract_to = '/content/eval_data'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Extracted all files to: {extract_to}")

Extracted all files to: /content/eval_data


In [8]:
def train_sequential_models():
    """Trains sequential models from D1 to D10 and evaluates on held-out datasets."""
    model_prototypes = []
    accuracies = []

    # Process datasets D1 to D10
    prototypes = None
    for i in tqdm(range(1, 11), desc="Processing Training Datasets"):
        data_path = f'train_data/train_data/{i}_train_data.tar.pth'
        dataset = torch.load(data_path)
        images, labels = dataset['data'], dataset.get('targets', None)

        # Feature extraction
        dataset = CustomImageDataset(images, labels, transform=transform)
        loader = DataLoader(dataset, batch_size=32, shuffle=False)
        features, lbls = extract_features(feature_extractor, loader, device)

        # Update prototypes
        if labels is not None:  # D1 is labeled
            prototypes = compute_prototypes(features, lbls)
        else:  # Unlabeled datasets
            predicted_labels = classify_with_prototypes(prototypes, features)
            prototypes = compute_prototypes(features, predicted_labels)

        model_prototypes.append(prototypes)

        # Evaluate on all held-out datasets up to current model
        model_accuracies = []
        for eval_idx in tqdm(range(1, i + 1), desc=f"Evaluating Model f{i}"):
            eval_data = torch.load(f'eval_data/eval_data/{eval_idx}_eval_data.tar.pth')
            eval_images, eval_labels = eval_data['data'], eval_data['targets']
            eval_dataset = CustomImageDataset(eval_images, eval_labels, transform=transform)
            eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

            eval_features, eval_lbls = extract_features(feature_extractor, eval_loader, device)
            predicted_labels = classify_with_prototypes(prototypes, eval_features)

            accuracy = (predicted_labels == eval_lbls).float().mean().item()
            model_accuracies.append(accuracy)
            print(f"Model f{i} accuracy on D̂{eval_idx}: {accuracy * 100:.2f}%")

        accuracies.append(model_accuracies)

    # Save the prototypes for f10
    prototype_save_path = "prototypes_f10.pth"
    torch.save(prototypes, prototype_save_path)
    print(f"Prototypes for f10 saved as {prototype_save_path}")

    return accuracies

In [9]:
# Main execution
if __name__ == "__main__":
    accuracy_matrix = train_sequential_models()
    print("\nAccuracy Matrix:")
    for row in accuracy_matrix:
        print(row)

Processing Training Datasets:   0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-8-ee33d4f4379a>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(dat

Model f1 accuracy on D̂1: 89.88%



Evaluating Model f2:  50%|█████     | 1/2 [00:20<00:20, 20.33s/it]

Model f2 accuracy on D̂1: 88.28%



Processing Training Datasets:  20%|██        | 2/10 [01:42<07:05, 53.15s/it]

...

Model f10 accuracy on D̂8: 87.56%



Evaluating Model f10:  90%|█████████ | 9/10 [03:03<00:20, 20.43s/it]

Model f10 accuracy on D̂9: 86.72%



Processing Training Datasets: 100%|██████████| 10/10 [22:06<00:00, 132.65s/it]

Model f10 accuracy on D̂10: 86.84%
Prototypes for f10 saved as prototypes_f10.pth

Accuracy Matrix:
[0.8988000154495239]
[0.8827999830245972, 0.8971999883651733]
[0.8772000074386597, 0.8939999938011169, 0.8827999830245972]
[0.873199999332428, 0.892799973487854, 0.8795999884605408, 0.8823999762535095]
[0.8736000061035156, 0.892799973487854, 0.8776000142097473, 0.8795999884605408, 0.8871999979019165]
[0.871999979019165, 0.8912000060081482, 0.8755999803543091, 0.8776000142097473, 0.8831999897956848, 0.8812000155448914]
[0.8744000196456909, 0.8899999856948853, 0.876800000667572, 0.8755999803543091, 0.8827999830245972, 0.8812000155448914, 0.8784000277519226]
[0.864799976348877, 0.8823999762535095, 0.8715999722480774, 0.871999979019165, 0.8744000196456909, 0.8772000074386597, 0.8668000102043152, 0.8736000061035156]
[0.8600000143051147, 0.8808000087738037, 0.8672000169754028, 0.8704000115394592, 0.8736000061035156, 0.8723999857902527, 0.8668000102043152, 0.8740000128746033, 0.8676000237464905